In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMG_WIDTH, IMG_HEIGHT = 224, 224
NUM_CLASSES = 10  # Change this to the number of classes in your dataset

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
# Uncomment the next line if using VGG19
# base_model = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Combine the base model and the top model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Set up data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train', target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory('validation', target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=32, class_mode='categorical')

# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)


58889256/58889256 [==============================] - 101s 2us/step


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'train'

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3, MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMG_WIDTH, IMG_HEIGHT = 224, 224
NUM_CLASSES = 10  # Number of classes in CIFAR-10

# Load the pre-trained InceptionV3 model
base_model_incep = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Load the pre-trained MobileNet model
base_model_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Create a custom top model for InceptionV3
x_incep = base_model_incep.output
x_incep = layers.GlobalAveragePooling2D()(x_incep)
x_incep = layers.Dense(1024, activation='relu')(x_incep)
x_incep = layers.Dense(512, activation='relu')(x_incep)
predictions_incep = layers.Dense(NUM_CLASSES, activation='softmax')(x_incep)

# Combine the base model and the top model for InceptionV3
model_incep = models.Model(inputs=base_model_incep.input, outputs=predictions_incep)

# Create a custom top model for MobileNet
x_mobilenet = base_model_mobilenet.output
x_mobilenet = layers.GlobalAveragePooling2D()(x_mobilenet)
x_mobilenet = layers.Dense(1024, activation='relu')(x_mobilenet)
x_mobilenet = layers.Dense(512, activation='relu')(x_mobilenet)
predictions_mobilenet = layers.Dense(NUM_CLASSES, activation='softmax')(x_mobilenet)

# Combine the base model and the top model for MobileNet
model_mobilenet = models.Model(inputs=base_model_mobilenet.input, outputs=predictions_mobilenet)

# Compile the models
model_incep.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summaries
model_incep.summary()
model_mobilenet.summary()

# Set up data generators for training and validation using CIFAR-10
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, tf.keras.utils.to_categorical(train_labels, NUM_CLASSES), batch_size=32)
validation_generator = test_datagen.flow(test_images, tf.keras.utils.to_categorical(test_labels, NUM_CLASSES), batch_size=32)

# Train the models
model_incep.fit(train_generator, epochs=1, validation_data=validation_generator)
model_mobilenet.fit(train_generator, epochs=1, validation_data=validation_generator)
